In [3]:
### Import the Libaries:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
### Open the Wikipedia page as URL:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

In [5]:
soup = BeautifulSoup(page, "lxml")

In [6]:
#print(soup.prettify())

In [7]:
### Find the table we are looking for:
toronto_table = soup.find("table" ,class_='wikitable sortable')
#toronto_table

In [8]:
### Create three empy lists to fill with the data from the toronto_table:
PC_list=[]
Borough_list=[]
Neighborhood_list=[]

### Loop trough the toronto_table and fill the three list with data:
for row in toronto_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        PC_list.append(cells[0].find(text=True))
        Borough_list.append(cells[1].find(text=True))
        Neighborhood_list.append(cells[2].find(text=True))

In [9]:
### Create the dataframe from the three lists:
df = pd.DataFrame(PC_list,columns=['PostalCode'])
df['Borough']=Borough_list
df['Neighborhood']=Neighborhood_list
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
5,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
6,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
7,M8A\n,Not assigned\n,\n
8,M9A\n,Etobicoke\n,"Islington Avenue, Humber Valley Village\n"
9,M1B\n,Scarborough\n,"Malvern, Rouge\n"


In [10]:
### Clean the \n form the dataframe:
df=df.replace({'\n': ''}, regex=True) 
### Clean the dataframe where the Borough is 'Not assigned':
df = df[df['Borough'] != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
df.shape

(103, 3)

In [53]:
### The geocode did not work so I will be using the Geospation file:
df_latlon = pd.read_csv('Geospatial_Coordinates.csv')
df_latlon.rename(columns = {'Postal Code' : 'PostalCode'}, inplace=True)
df_latlon.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [65]:
### Now we merge the two dataframes on 'PostalCode':
df_new = pd.merge(df, df_latlon, left_on='PostalCode', right_on='PostalCode')


In [68]:
### Drop the second Postal Code row:
df_new.drop('Postal Code', axis=1)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
